# 🚀 A100 AGGRESSIVE - DINOv2 完整 5-Fold 訓練

**GPU要求**: A100 (40GB)
**Batch Size**: 24 (激進)
**預計時間**: 2-3 小時 (完整 5-Fold)
**目標分數**: 90%+

---

## ⚡ 設置步驟
1. Runtime → Change runtime type → **A100 GPU**
2. 準備 `kaggle.json` (Kaggle → Account → Create API Token)
3. 順序執行所有 Cell

---

## 1️⃣ 環境檢查

In [ ]:
import torch
import os

print("="*70)
print("🔍 GPU 檢查")
print("="*70)

if not torch.cuda.is_available():
    raise RuntimeError("❌ No GPU! 請啟用 A100: Runtime → Change runtime type → GPU")

gpu_name = torch.cuda.get_device_name(0)
gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9

print(f"✅ GPU: {gpu_name}")
print(f"✅ VRAM: {gpu_memory:.1f} GB")

if "A100" not in gpu_name:
    print(f"\n⚠️ 警告: 檢測到 {gpu_name}，不是 A100！")
    print("   本 Notebook 針對 A100 優化，其他 GPU 請使用對應版本")
    raise RuntimeError("Please use A100 GPU for this notebook")

# A100 激進配置
BATCH_SIZE = 24
EXPECTED_TIME_PER_FOLD = "25-30 分鐘"
TOTAL_TIME = "2-2.5 小時"

print(f"\n🚀 A100 激進配置:")
print(f"   Batch Size: {BATCH_SIZE}")
print(f"   預計時間/fold: {EXPECTED_TIME_PER_FOLD}")
print(f"   預計總時間: {TOTAL_TIME}")
print("="*70)

## 2️⃣ 安裝依賴

In [ ]:
%%capture
!pip install -q timm>=1.0.0 pyyaml kaggle scikit-learn tqdm

import timm
print(f"✅ timm: {timm.__version__}")

dinov2_models = [m for m in timm.list_models() if 'dinov2' in m and 'base' in m]
print(f"✅ DINOv2 models: {dinov2_models}")

## 3️⃣ Kaggle API

In [ ]:
from google.colab import userdata
import json

print("🔑 從 Colab Secrets 讀取 Kaggle API...")

# 從 Colab Secrets 讀取 API key
try:
    kaggle_key = userdata.get('KAGGLE_KEY')
    print("✅ 成功讀取 KAGGLE_KEY")
except Exception as e:
    print("❌ 無法讀取 KAGGLE_KEY")
    print("請在左側邊欄 🔑 Secrets 中添加:")
    print("   名稱: KAGGLE_KEY")
    print("   值: 你的 Kaggle API key")
    raise e

# 創建 kaggle.json
kaggle_config = {
    "username": "thc1006",
    "key": kaggle_key
}

!mkdir -p ~/.kaggle

with open('/root/.kaggle/kaggle.json', 'w') as f:
    json.dump(kaggle_config, f)

!chmod 600 ~/.kaggle/kaggle.json

print("✅ Kaggle API 配置完成 (username: thc1006)")

## 4️⃣ Clone GitHub

In [ ]:
REPO_URL = "https://github.com/thc1006/nycu-CSIC30014-LAB3.git"
REPO_NAME = "nycu-CSIC30014-LAB3"

if os.path.exists(REPO_NAME):
    !cd {REPO_NAME} && git pull
else:
    !git clone {REPO_URL}

%cd {REPO_NAME}
print(f"✅ 當前目錄: {os.getcwd()}")

## 5️⃣ 下載數據

In [ ]:
print("📥 下載 Kaggle 數據 (~700MB)...")
!kaggle competitions download -c cxr-multi-label-classification
!unzip -q cxr-multi-label-classification.zip -d data/
!rm cxr-multi-label-classification.zip

# 驗證數據
import glob
train_images = glob.glob('data/train_images/*.jpg') + glob.glob('data/train_images/*.jpeg') + glob.glob('data/train_images/*.png')
test_images = glob.glob('data/test_images/*.jpg') + glob.glob('data/test_images/*.jpeg') + glob.glob('data/test_images/*.png')

print(f"✅ 訓練影像: {len(train_images)}")
print(f"✅ 測試影像: {len(test_images)}")
print(f"✅ CSV 檔案已從 GitHub 同步")

# Kaggle 競賽只有 train_images 和 test_images
# train/val split 由 CSV 控制
assert len(test_images) == 1182, f"測試集不完整: {len(test_images)} != 1182"

## 6️⃣ 準備 K-Fold 數據

In [ ]:
import pandas as pd

# Kaggle 競賽數據結構: 只有 train_images/ 和 test_images/
# fold CSV 已從 GitHub 同步，直接使用
print("📋 驗證 Fold CSV...")

for fold in range(5):
    train_csv = f'data/fold{fold}_train.csv'
    val_csv = f'data/fold{fold}_val.csv'
    
    if not os.path.exists(train_csv) or not os.path.exists(val_csv):
        print(f"❌ Fold {fold} CSV 不存在，需要重新生成")
        raise FileNotFoundError(f"Missing fold CSV for fold {fold}")
    
    train_df = pd.read_csv(train_csv)
    val_df = pd.read_csv(val_csv)
    
    # 修正 source_dir: Kaggle 所有訓練影像都在 train_images/
    train_df['source_dir'] = 'data/train_images'
    val_df['source_dir'] = 'data/train_images'  # ✅ Kaggle 結構
    
    # 保存修正後的 CSV
    train_df.to_csv(train_csv, index=False)
    val_df.to_csv(val_csv, index=False)
    
    print(f"✅ Fold {fold}: Train={len(train_df)}, Val={len(val_df)}")

print("\n✅ Fold 數據準備完成 (已適配 Kaggle 結構)")

## 7️⃣ 完整 5-Fold 訓練 (A100 激進模式)

In [ ]:
import time

EPOCHS = 35
BATCH_SIZE = 24  # A100 激進
IMG_SIZE = 518
LR = 3e-5
OUTPUT_DIR = "outputs/dinov2_a100_aggressive"

print("="*70)
print("🚀 A100 激進訓練開始")
print("="*70)
print(f"Batch Size: {BATCH_SIZE}")
print(f"預計總時間: 2-2.5 小時")
print("="*70)

start_time = time.time()

for fold in range(5):
    fold_start = time.time()
    
    print(f"\n{'='*70}")
    print(f"📊 Fold {fold}/4 訓練中...")
    print(f"{'='*70}\n")
    
    !python3 train_dinov2_breakthrough.py \
        --fold {fold} \
        --epochs {EPOCHS} \
        --batch_size {BATCH_SIZE} \
        --img_size {IMG_SIZE} \
        --lr {LR} \
        --output_dir {OUTPUT_DIR}
    
    fold_time = (time.time() - fold_start) / 60
    total_time = (time.time() - start_time) / 60
    
    print(f"\n✅ Fold {fold} 完成: {fold_time:.1f} 分鐘")
    print(f"   累計: {total_time:.1f} 分鐘 ({total_time/60:.1f} 小時)")
    
    if fold < 4:
        remaining = fold_time * (4 - fold)
        print(f"   預估剩餘: {remaining:.1f} 分鐘 ({remaining/60:.1f} 小時)")

total_time = (time.time() - start_time) / 60
print(f"\n{'='*70}")
print(f"🎉 全部訓練完成！")
print(f"總耗時: {total_time:.1f} 分鐘 ({total_time/60:.2f} 小時)")
print(f"{'='*70}")

## 8️⃣ 生成 5-Fold 集成預測

In [ ]:
import numpy as np
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as T
from tqdm.auto import tqdm
import timm

class TestDataset(Dataset):
    def __init__(self, csv_path, img_dir, img_size=518):
        self.df = pd.read_csv(csv_path)
        self.img_dir = img_dir
        self.transform = T.Compose([
            T.Resize((img_size, img_size)),
            T.ToTensor(),
            T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_dir, row['new_filename'])
        img = Image.open(img_path).convert('RGB')
        return self.transform(img), row['new_filename']

# 準備測試集
test_dataset = TestDataset('data/test_data_sample.csv', 'data/test_images', img_size=518)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2, pin_memory=True)

device = torch.device('cuda')
print(f"✅ 測試集: {len(test_dataset)} 樣本\n")

# 收集所有 fold 預測
all_preds = []
filenames = None

for fold in range(5):
    model_path = f'{OUTPUT_DIR}/fold{fold}/best.pt'
    
    if not os.path.exists(model_path):
        print(f"⚠️ Fold {fold} 模型不存在，跳過")
        continue
    
    print(f"📊 Fold {fold} 預測中...")
    
    # 載入模型 (修正: 使用正確的模型名稱和載入方式)
    model = timm.create_model('vit_base_patch14_dinov2', pretrained=False, num_classes=4)
    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])  # ✅ 修正
    model = model.to(device)
    model.eval()
    
    fold_probs = []
    fold_filenames = []
    
    with torch.no_grad():
        for images, fnames in tqdm(test_loader, desc=f'Fold {fold}', leave=False):
            images = images.to(device)
            outputs = model(images)
            probs = torch.softmax(outputs, dim=1)
            fold_probs.append(probs.cpu().numpy())
            fold_filenames.extend(fnames)
    
    fold_probs = np.concatenate(fold_probs, axis=0)
    all_preds.append(fold_probs)
    
    if filenames is None:
        filenames = fold_filenames
    
    print(f"   ✅ 完成\n")

# 集成
print(f"🔮 集成 {len(all_preds)} 個模型...")
avg_probs = np.mean(all_preds, axis=0)
final_preds = np.argmax(avg_probs, axis=1)

# 創建提交
class_names = ['normal', 'bacteria', 'virus', 'COVID-19']
submission_df = pd.DataFrame({
    'new_filename': filenames,
    'label': [class_names[p] for p in final_preds]
})

submission_path = f'submission_a100_aggressive_{len(all_preds)}fold.csv'
submission_df.to_csv(submission_path, index=False)

print(f"\n✅ 提交文件已保存: {submission_path}")
print(f"\n預測分布:")
print(submission_df['label'].value_counts())

## 9️⃣ 提交到 Kaggle

In [ ]:
import glob

submission_files = glob.glob('submission_a100_*.csv')
if submission_files:
    latest = max(submission_files, key=os.path.getctime)
    
    print(f"📤 提交: {latest}\n")
    
    !kaggle competitions submit \
        -c cxr-multi-label-classification \
        -f {latest} \
        -m "DINOv2 5-Fold A100 Aggressive (BS=24) - Target 90%+"
    
    print("\n" + "="*70)
    print("✅ 提交完成！")
    print("="*70)
    
    print("\n最近提交:")
    !kaggle competitions submissions -c cxr-multi-label-classification | head -10
else:
    print("❌ 找不到提交文件")

---

## ✅ 完成

**預期結果**: 89.5-91.5% Macro-F1

**總時間**: ~2-2.5 小時 (A100)

**GitHub**: https://github.com/thc1006/nycu-CSIC30014-LAB3